## **Spotify Recommender - Part 2**

The fist step is to insall the needed libraries and mount the drive to upload the files with the streaming histories and their punctuations:



In [1]:
!pip3 install numpy
!pip3 install scikit-surprise

#!pip install -q scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 8.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633745 sha256=1b709a6b7af32a7b8a0ba210c6c3b5ce507d0e8a053a569ffdd83ba6d43c9360
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from google.colab import drive
from surprise import BaselineOnly
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go


from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering


                          
                                               
from surprise import accuracy                                              
#from surprise import GridSearch 

import math
import re
import matplotlib.pyplot as plt
import seaborn as sns
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Upload the streaming histories and create a dataframe containg the titles of the songs with a number asigned to them, to later replace the titles for the codes at the main data frame

In [3]:
df_users=pd.read_csv('/content/drive/MyDrive/Master/Q2/Machine_learning/Final_Project/fitxers_importantissims/output0_23.csv')

##Això es pq sortien columnes rarissimes:

df_treball=pd.DataFrame(columns=['trackName','user', 'item', 'rating'])
df_treball['trackName']=df_users['trackName']
df_treball['rating']=df_users['Punctuation']
df_treball['user']=df_users['User']


#cal posar un pd que les songs estiguin amb un id
songs=df_treball['trackName'].unique().tolist()
songs_num=np.arange(0,  len(songs), 1)
df_songs= pd.DataFrame(columns=['songs', 'song_num'])
df_songs['songs']=songs
df_songs['song_num']=songs_num
#df_songs.head()
##fer servir això per per fer un replace i dexar el df principal amb els valors
songs_id=[]
for i in range(len(df_treball)):
  song_df=df_treball.iat[i,0]
  index=df_songs.loc[df_songs['songs']==song_df]['song_num'].tolist()[0]
  songs_id.append(index)
df_treball['item']=songs_id
df_treball=df_treball.drop(columns='trackName') 
df_treball.head()

,user,item,rating
0,0,0,5
1,0,1,5
2,0,2,5
3,0,3,5
4,0,4,4


In [ ]:
from google.colab import files

df_treball.to_csv('df_treball.csv', encoding = 'utf-8-sig') 
files.download('df_treball.csv')

df_songs.to_csv('df_songs.csv', encoding = 'utf-8-sig') 
files.download('df_songs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

To check how many ratings have each song, a histogram is used.


In [ ]:
df=df_treball
# Number of ratings per book
data_plot = df.groupby('item')['rating'].count().clip(upper=100)

# Create trace
trace = go.Histogram(x = data_plot.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per Item (Clipped at 100)',
                   xaxis = dict(title = 'Number of Ratings Per Item'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.show()

To have more information about the data we are working, specially the punctuation that we've assigned in the other code, we check how the puntuation is distributed. 

In [ ]:
df=df_treball
p = df.groupby('rating')['rating'].agg(['count'])

item_count = df["item"].nunique()
user_count = df['user'].nunique()
rating_count = df["rating"].count()

ax = p.plot(kind = 'barh', legend = False, figsize = (15,10))
plt.title('Total pool: {:,} Songs, {:,} users, {:,} ratings given'.format(item_count, user_count, rating_count), fontsize=20)
plt.axis('off')
          
for i in range(1,6):
    ax.text(p.iloc[i-1][0]/4, i-1, 'Rating {}: {:.0f}%'.format(i, p.iloc[i-1][0]*100 / p.sum()[0]), color = 'black', weight = 'bold')

**Evaluating predictions**

To evaluate which predictor method is better, we use Root Mean Square Error (RMSE) as the accuracy metric for the predictions. 

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)

benchmark = []
# Iterate over all algorithms

algorithms = [SVD(), NormalPredictor(),  KNNBasic(), KNNWithMeans(), KNNBaseline(), KNNWithZScore()]
#, SVDpp()
print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')

Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f0c96a53e50>, <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7f0c96a53110>, <surprise.prediction_algorithms.knns.KNNBasic object at 0x7f0c96a53150>, <surprise.prediction_algorithms.knns.KNNWithMeans object at 0x7f0c96a531d0>, <surprise.prediction_algorithms.knns.KNNBaseline object at 0x7f0c96a53510>, <surprise.prediction_algorithms.knns.KNNWithZScore object at 0x7f0c96b05f50>] 



Starting:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f0c96a53e50>
Done:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f0c96a53e50> 


Starting:  <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7f0c96a53110>
Done:  <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7f0c96a53110> 


Starting:  <surprise.prediction_algorithms.knns.KNNBasic object at 0x7f0c96a53150>
Computing the msd similarity matrix...
Done 

In [ ]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.544165,4.069339,0.352387
KNNBaseline,0.619591,0.366013,1.733372
KNNWithMeans,0.651112,0.077220,1.385175
KNNWithZScore,0.653007,0.124882,1.576303
KNNBasic,0.653042,0.053884,1.325960
NormalPredictor,1.165525,0.115663,0.282507


The method that shows less error is the SVD so it's the method that will be used. THe SVDpp took too much time to evaluate

**Evaluating the best paraters**


In [6]:
# param_grid = {
#     "n_epochs": [5, 10, 15, 20, 30, 40, 50, 100],
#     "lr_all": [0.001, 0.002, 0.005],
#     "reg_all": [0.02, 0.08, 0.4, 0.6]
# }

# smaller grid for testing and having a result
param_grid = {
    "n_epochs": [ 5, 10, 20, 30],
    "lr_all": [0.001, 0.002, 0.005],
    "reg_all": [0.001, 0.02]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)

gs.fit(data)

training_parameters = gs.best_params["rmse"]

print("BEST RMSE: \t", gs.best_score["rmse"])
print("BEST MAE: \t", gs.best_score["mae"])
print("BEST params: \t", gs.best_params["rmse"])

BEST RMSE: 	 0.476554159622612
BEST MAE: 	 0.16585585940872719
BEST params: 	 {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.001}


**Recommendation with Collaborative Filtering**

This project utilizes collaborative filtering, a system matching persons with similar interests and providing recommendations based on this matching. Collaborative filters do not require item metadata like its content-based counterparts.

Surprise library is utilized here to implement SVD

In [11]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(usuari, predictions, n=10):
  

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    
    #"aqui printeem resultats
  
    cancons=top_n[usuari]   
    recomenacio=[]
    for i in range(len(cancons)):
      codi=cancons[i][0]
      row=df_songs.loc[df_songs['song_num']==codi]
      canco=row['songs'].to_list()[0]
      recomenacio.append(canco)
    print("For user {} the {} recommended songs are: ".format(usuari, n))
    for x in range(len(recomenacio)):
      print("  {} - {}".format(x, recomenacio[x]))
    print("Enjoy!")
    return top_n


# First train an SVD algorithm on the movielens dataset.
df=df_treball
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)

trainset = data.build_full_trainset()
trainset, testset = train_test_split(data, test_size=.25)
algo = SVD()

#Prva parametres:

param_grid = {
    "n_epochs": [20],
    "lr_all": [0.005],
    "reg_all": [0.001] }

gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)

gs.fit(data)

training_parameters = gs.best_params["rmse"]

algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
#predictions = algo.fit(trainset).test(testset)
predictions = algo.test(testset)

###################################
#      INPUT
###################################

usuari = 18
n=8
top_n = get_top_n(usuari,predictions, n)

#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


For user 18 the 8 recommended songs are: 
  0 - Qué Más Pues?
  1 - LA CANCIÓN
  2 - Si Quieren Frontear
  3 - Las Nenas (feat. La Duraca)
  4 - Fruit Punch
  5 - Not Afraid
  6 - That Girl
  7 - Hold (Candyland Remix)
Enjoy!
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5256  0.5203  0.5294  0.5317  0.5258  0.5266  0.0039  
MAE (testset)     0.2661  0.2650  0.2669  0.2693  0.2667  0.2668  0.0014  
Fit time          5.04    5.02    5.03    5.08    5.13    5.06    0.04    
Test time         0.14    0.22    0.19    0.15    0.16    0.17    0.03    


{'fit_time': (5.03706955909729,
  5.024122714996338,
  5.033466577529907,
  5.080150842666626,
  5.134284019470215),
 'test_mae': array([0.26614056, 0.26495097, 0.2668633 , 0.26927285, 0.26669512]),
 'test_rmse': array([0.52564023, 0.52028922, 0.52936265, 0.53172344, 0.52584041]),
 'test_time': (0.14062261581420898,
  0.22325587272644043,
  0.18905162811279297,
  0.1458897590637207,
  0.1570122241973877)}